In [ ]:
import pandas as pd
import numpy as np
import os

from datetime import datetime

import yfinance as yf
import plotly.graph_objects as go

from positions import Positions

In [ ]:
file = "eurusd-5m"

col_names = ['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume']

raw_path = os.path.join(os.getcwd(), "../d2", f'{file}.csv')
updated_file_path = os.path.join(os.getcwd(), "../d2/converted", f'{file}-updated.csv')

exists = os.path.isfile(updated_file_path)

args = {}

if exists: 
    args['parse_dates'] = True
    args['index_col'] = 'Datetime'
    
else:
    args['sep'] = ';'
    args['names'] = col_names
    header=None

print(exists, raw_path, updated_file_path)

ticker_data = pd.read_csv(
    updated_file_path if exists else raw_path , **args)


def convert_date(row):
    # print(row['Date']+' '+row['Time'])

    row["Datetime"] = datetime.strptime(
        row["Date"] + " " + row["Time"], "%d/%m/%Y %H:%M:%S"
    )

    return row

if not exists: 
    ticker_data = ticker_data.apply(convert_date, axis=1)
    ticker_data.to_csv(updated_file_path)

ticker_data

In [ ]:
class Backtest:
    balance = 0
    margin_used = 0
    start_balance = 0

    stoploss = 0
    target = 0

    positions = Positions()

    trade_book = pd.DataFrame(
        columns=[
            "entry_date",
            "entry_price",
            "exit_price",
            "exit_date",
            "target",
            "stoploss",
            "PnL",
            "crossover_type",
            "active",
            "size",
            "margin_used",
            "balance",
            "balance_on_open",
        ]
    )

    def __init__(self, balance, stoploss, reward_ratio):
        self.start_balance = balance
        self.balance = balance
        self.stoploss = stoploss
        self.target = stoploss * reward_ratio

        self.positions.clear_positions()
        self.reset_tradebook()

    def clear_position(self, position):
        self.positions.delete_position(position)

    def reset_tradebook(self):
        self.trade_book = pd.DataFrame(
            columns=[
                "entry_date",
                "entry_price",
                "exit_price",
                "exit_date",
                "target",
                "stoploss",
                "PnL",
                "crossover_type",
                "active",
                "size",
                "margin_used",
                "balance",
                "balance_on_open",
            ]
        )

    def open(self, entry_date, entry_price, stoploss, target, size, crossover_type):
        if self.balance is None or self.balance == 0:
            raise Exception("Oops, you ran out of bucks.")

        if self.balance < entry_price * size:
            print("Balance: ", self.balance)
            print("Lot Size: ", size)
            print("Ticket Price: ", entry_price * size)
            raise Exception("Oops, you are low on bucks.")

        position = {
            "entry_date": entry_date,
            "entry_price": entry_price,
            "stoploss": stoploss,
            "target": target,
            "size": size,
            "crossover_type": crossover_type,
            "margin_used": entry_price * size,
            "balance_on_open": self.balance - (entry_price * size),
        }

        self.margin_used = self.margin_used + (entry_price * size)

        self.positions.add_position(position)

        self.balance = self.balance - entry_price * size

    def close(self, position, candle, exit_price):
        # Open position is now closed & logged into the trade book
        trade = {
            **position,
            "exit_date": candle.name,
            "exit_price": exit_price,
        }

        trade["PnL"] = (exit_price - position["entry_price"]) * position["size"]

        if position["crossover_type"] == "short":
            trade["PnL"] = -1 * trade["PnL"]

        balance = self.balance + position["margin_used"] + trade["PnL"]
        trade["balance"] = balance
        self.balance = balance

        row = pd.Series(trade)
        self.trade_book.loc[len(self.trade_book)] = row

        self.positions.delete_position(position)

In [ ]:
class Tester(Backtest):
    ticker_data = None
    count = 0
    close_count = 0

    def __init__(self, balance, stoploss, reward_ratio):
        super().__init__(balance, stoploss, reward_ratio)
        self.ticker_data = None
        self.reset_tradebook()

        self.profitable_trades = None
        self.losing_trades = None

    def init(self, ticker_data, size, small_ema, large_ema):
        ticker_data["small_ema"] = (
            ticker_data["Close"].ewm(span=small_ema, adjust=False).mean()
        )

        ticker_data["large_ema"] = (
            ticker_data["Close"].ewm(span=large_ema, adjust=False).mean()
        )

        ticker_data["crossover_pos"] = np.where(
            ticker_data["small_ema"] > ticker_data["large_ema"], 1, 0
        )

        ticker_data["crosspoint"] = np.where(
            ticker_data["crossover_pos"] != ticker_data["crossover_pos"].shift(1),
            True,
            False,
        )

        self.small_ema = small_ema
        self.large_ema = large_ema
        self.ticker_data = None
        self.ticker_data = ticker_data
        self.size = size

    def chart(self, small_ema, large_ema):
        fig = go.Figure(
            data=[
                go.Candlestick(
                    x=ticker_data.index,
                    open=ticker_data["Open"],
                    high=ticker_data["High"],
                    low=ticker_data["Low"],
                    close=ticker_data["Close"],
                ),
                go.Scatter(
                    x=ticker_data.index,
                    y=ticker_data["small_ema"],
                    name=f"small_ema_{small_ema}",
                ),
                go.Scatter(
                    x=ticker_data.index,
                    y=ticker_data["large_ema"],
                    name=f"large_ema_{large_ema}",
                ),
            ]
        )

        fig.update_layout(xaxis_rangeslider_visible=False)
        fig.show()

    def print(self):
        print(self.position)

    def run(self):
        if self.ticker_data is None:
            raise Exception("No ticker data to run backtest on")

        # Skip first 200 candles
        start_index = self.large_ema

        while start_index < len(self.ticker_data):
            self.strategy(self.ticker_data.iloc[start_index])
            start_index += 1

    def stats(self):
        print("Small EMA: ", self.small_ema, "\n")
        print("Large EMA: ", self.large_ema, "\n")
        print("Total Trades: ", len(self.trade_book), "\n")
        print("PnL: ", self.trade_book["PnL"].sum(), "\n")
        print("Initial Balance: ", self.start_balance, "\n")
        print("Trade Size: ", self.size, "\n")
        print("Stoploss: ", self.stoploss, "\n")
        print("Target: ", self.target, "\n")

        crosspoints = len(
            self.ticker_data[self.large_ema :][self.ticker_data["crosspoint"] == True]
        )
        total_trades = len(self.trade_book)
        profitable_trades = self.trade_book[self.trade_book["PnL"] > 0]
        losing_trades = self.trade_book[self.trade_book["PnL"] < 0]
        win_percentage = len(profitable_trades) / total_trades * 100
        lose_percentage = len(losing_trades) / total_trades * 100

        print("Profitable Trades: ", len(profitable_trades), "\n")
        print("Losing Trades: ", len(losing_trades), "\n")
        print("Win Percentage: ", win_percentage, "\n")
        print("Lose Percentage: ", lose_percentage, "\n")
        print(
            "Date Range: ",
            self.trade_book["entry_date"].iloc[0],
            " - ",
            self.trade_book["exit_date"].iloc[-1],
            "\n",
        )
        print("Time Period: ", period)
        print("Crosspoints: ", crosspoints, "\n")
        print("Open Positions / Trades: ", self.positions.count(), "\n")
        print("Open Counter: ", self.count, "\n")
        print("Close Counter: ", self.close_count, "\n")

        self.profitable_trades = profitable_trades
        self.losing_trades = losing_trades
        self.crosspoints = crosspoints

    def dump_stats(self):
        columns = [
            "ticker",
            "period",
            "crosspoints",
            "interval",
            "small_ema",
            "large_ema",
            "total_trades",
            "pnl",
            "initial_balance",
            "trade_size",
            "stoploss",
            "target",
            "profitable_trades",
            "losing_trades",
            "win_percentage",
            "lose_percentage",
            "date_range",
        ]

        total_trades = len(self.trade_book)
        profitable_trades = self.trade_book[self.trade_book["PnL"] > 0]
        losing_trades = self.trade_book[self.trade_book["PnL"] < 0]
        win_percentage = len(profitable_trades) / total_trades * 100
        lose_percentage = len(losing_trades) / total_trades * 100

        stats = {
            "ticker": ticker,
            "period": period,
            "crosspoints": self.crosspoints,
            "interval": interval,
            "small_ema": self.small_ema,
            "large_ema": self.large_ema,
            "total_trades": len(self.trade_book),
            "pnl": self.trade_book["PnL"].sum(),
            "initial_balance": self.start_balance,
            "trade_size": self.size,
            "stoploss": self.stoploss,
            "target": self.target,
            "profitable_trades": len(profitable_trades),
            "losing_trades": len(losing_trades),
            "win_percentage": win_percentage,
            "lose_percentage": lose_percentage,
            "date_range": f'{self.trade_book["entry_date"].iloc[0]} - {self.trade_book["exit_date"].iloc[-1]}',
        }

        df = pd.DataFrame(stats, index=[0])
        df.to_csv("dump_datafiles.csv", mode="a", index=False, header=False)

    def check_position(self, position, candle):
        if position["crossover_type"] == "long":
            if (
                candle["Low"] < position["stoploss"]
                and position["stoploss"] > candle["Close"]
            ):
                return candle["Close"]

            if candle["High"] > position["target"]:
                return position["target"]

        if position["crossover_type"] == "short":
            if (
                candle["High"] > position["stoploss"]
                and position["stoploss"] < candle["Close"]
            ):
                return candle["Close"]

            if candle["Low"] < position["target"]:
                return position["target"]

        return None

    def strategy(self, candle):
        if self.positions.count() > 0:
            # check if target met or stoploss hit
            # calculate trade results, PnL
            # close position

            for position in self.positions.get_positions():
                exit_price = self.check_position(position, candle)

                if exit_price is not None:
                    self.close_count = self.close_count + 1

                    self.close(position, candle, exit_price)

        if candle["crosspoint"] == True:
            entry_price = candle["Close"]
            crossover_type = "long" if candle["crossover_pos"] == 1 else "short"
            stoploss = None
            target = None

            self.count = self.count + 1

            if crossover_type == "short":
                stoploss = entry_price + self.stoploss
                target = entry_price - self.target

            if crossover_type == "long":
                stoploss = entry_price - self.stoploss
                target = entry_price + self.target

            self.open(
                entry_date=candle.name,
                entry_price=entry_price,
                stoploss=stoploss,
                target=target,
                size=self.size,
                crossover_type=crossover_type,
            )

In [ ]:
small_ema =8
large_ema = 55

# balance, stoploss, reward_ratio
ttr = Tester(90_000_000, 0.009 , 10)
# ticker_data, ticker_size,  ticket_size, small_ema, large_ema
ttr.init(ticker_data, 100_000, small_ema, large_ema)
# small_ema, large_ema
# ttr.chart(small_ema, large_ema) 
# run backtesting
ttr.run()
ttr.stats()
ttr.dump_stats()

t_book2 = ttr.trade_book
profitables = ttr.profitable_trades
losers = ttr.losing_trades